# Quicksort

**Quicksort** is a recursive method:

- Shuffles the array
- Creates a partition so there is a `j` where entry `a[j]` is in place, there's no larger entry to the left of `j`, and no smaller entry to the right of `j`
- Recursively sort the left part and right part

Phase I (repeat until `i` and `j` cross:
- Choose the first element as `a[lo]`
- Scan `i` from left to right as long as `a[i] < a[lo]`
- Scan `j` from right to left so long as `a[j] > a[lo]`
- Exchange `a[i]` with `a[j]`

Phase II is to take each half around the partition and run phase I on it (using the first item as the new partition).

Typically partition in place - you can use an extra array to make it easier (and stable), but it's usually no worth the cost. One of the advantages of it over mergesort is that it's in place.

Two tricky parts are keeping the indices `i` and `j` from crossing (terminating the loop) and staying in bounds (`j == lo` test is redundant but `i == hi` isn't). The shuffling part is necessary for performance guarantees. Also, when there are duplicate keys, it's better to stop on keys equal to the partitioning item's key.

Quicksort is even faster than mergesort.

In the best case, quicksort will divide the array in half with each partition, and is then similar to mergesort with $N \lg N$. The worst case is if the array is already sorted, so each partition just peals off the first item, which is ~$\frac{1}{2} N^2$. But with random shuffling, that's highly unlikely to happen.

The average case (by number of compares) is more interesting.

The proposition is the average number of compares $C_N$ to quicksort an array of $N$ distinct keys is ~$2N \ln N$ (and the number of exchanges is ~$\frac{1}{3} N \ln N$).

**The proof**: $C_N$ satisfies the recurrence $C_0 = C_1 = 0$ and for $N \geq 2$. The following shows $C_N$ equal to the partitioning plus each left + right over the partitioning probability:

$$
C_N = (N + 1) + \bigg{(} \frac{C_0 + C_{N-1}}{N} \bigg{)} + \bigg{(} \frac{C_1 + C_{N-2}}{N} \bigg{)} + \cdots + \bigg{(} \frac{C_{N-1} + C_0}{N} \bigg{)} \\
\text{Multiply both sides by } N \\
NC_N = N(N + 1) + 2(C_0 + C_1 + \ldots + C_{N-1}) \\
\text{Subtract this from the same equation for } N-1: \\
NC_N - (N - 1)C_{N-1} = 2N + 2C_{N-1} \\
\text{Repeatedly apply the above equation, get approximate sum by integral:} \\
C_N \text{~} 2(N + 1) \ln N \approx 1.39 N \lg N
$$

**Worst case**, the number of compares is quadratic ($N^2$), but that's extremely unlikely with a random shuffle (it's a probabilistic guarantee against the worst case, and important for performance). Many text book implementations go quadratic if the array is already sorted/reverse sorted, or there are many duplicates. There are about 39% more compares than mergesort, but it's faster in practice because of less data movement.

**Summary of properties**:

- **In place**: partitioning is done with constant extra space
- **Depth of recursion**: can guarantee logarithmic depth by recurring on smaller subarrays before larger subarrays, so get logarithmic extra space with high probability
- **Not stable**: long-range swaps can pull items out of relative order

Practical improvements:

- Quicksort has too much overhead for small subarrays, can use insertion sort for arrays with ~10 items, or delay using it until one pass at the end
- Estimate the partitioning item instead of using the first item. The best choice for the pivot item is the median, can find a decent estimate by taking 3 random items and taking the median of that (improves performance by ~10%)

In [37]:
def partition(arr, lo=0, hi=None):
    """
    Performs one round of partitioning on the given array
    Inputs: arr is a list, lo and hi are integers representing indices
    Output: the index where items to left are < item, items to the right
        are > item
    Partition swaps are done in place
    """
    if hi is None:
        hi = len(arr)

    pivot = arr[hi - 1]
    i = lo
    for j in range(lo, hi):
        if arr[j] < pivot:
            arr[i], arr[j] = arr[j] , arr[i]
            i += 1
    arr[i], arr[hi - 1] = arr[hi - 1], arr[i]
    return i


def qsort(arr, lo=0, hi=None):
    """
    Quicksort implementation; performs recursive calls on subarrays
        after each round of partitioning it
    Inputs: arr is a list, lo and hi are integers representing indices
    Output: None, sort done in place
    """
    if hi is None:
        hi = len(arr)
    if hi - lo < 2:
        return
    pivot = partition(arr, lo, hi)

    qsort(arr, lo, pivot)
    qsort(arr, pivot + 1, hi)

    
def quicksort(arr):
    """
    Wrapper for qsort that first shuffles the array
    Input: arr is a list
    Output: None, sort done in place
    """
    import random
    random.shuffle(arr)
    qsort(arr)


a = list("QUICKSORTEXAMPLE")
print("ARRAY: {}".format(a))
quicksort(a)  # Sorting done in place
print("Quicksort results: {}".format(a))
nums = list(range(1, 11))
quicksort(nums)
print(nums)

ARRAY: ['Q', 'U', 'I', 'C', 'K', 'S', 'O', 'R', 'T', 'E', 'X', 'A', 'M', 'P', 'L', 'E']
Quicksort results: ['A', 'C', 'E', 'E', 'I', 'K', 'L', 'M', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'X']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Selection

The **selection** problem is related to sorting - the goal is to find the $k^{th}$ largest item out of a set of $N$ items. The minimum would be $k = 0$, maximum would be $k = N - 1$, and the median would be $k = N / 2$.

Is there a linear algorithm for every $k$?

You can use a version similar to quicksort: partition the array so that entry `a[j]` is in place, there's no larger entry to the left of `j` and no smaller entry to the right of `j`. Then repeat in one subarray, depending on `j`, finished when `j` equals `k`.

The number of compares are linear - which is the main advantage of quick select over quicksort.

In [46]:
def quickSelect(arr, k):
    """
    Returns the kth largest item out of array a
    Inputs: arr is a list, k is an integer (which
        kth-smallest item to return)
    Output: the kth-smallest item to return. k=0 is the min
        k=len(arr)-1 is the max
    """
    import random
    random.shuffle(arr)
    print("Shuffled array: {}".format(arr))
    
    lo = 0
    hi = len(arr)
    
    while hi > lo:
        j = partition(arr, lo, hi)
        if j < k:
            lo = j + 1
        elif j > k:
            hi = j  # Not j - 1 because partition doesn't include it
        else:
            return arr[k]
    
    return arr[k]

b = list(range(10))
k = 6
print("Find {}-smallest item in {}".format(k, b))
print("Item: {}".format(quickSelect(b, k)))

Find 6-smallest item in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Shuffled array: [0, 3, 5, 8, 6, 9, 4, 7, 2, 1]
Item: 6


## Duplicate Keys

The purpose of sorting is often to bring items with similar keys together, and you may have a huge array with a small number of key values.

**Mergesort with duplicate keys**: between $\frac{1}{2} N \lg N$ and $N \lg N$ compares.

**Quicksort with duplicate keys**: algorithm goes **quadratic** unless partitioning stops on equal keys!

One mistake is to put all items equal to the partitioning item on one side, but that leads to ~$\frac{1}{2} N^2$ compares when all keys are equal.

The recommended way to handle duplicate keys is to stop scans on items equal to the partitioning item. This leads to ~$N \lg N$ compares. You want the algorithm to put all items equal to the partitioning item in place.

This bug was found by a user in the C library's `qsort()` in 1990, and led to **3-way partitioning** approach (Dutch national flag problem, solved by Edsger Dijkstra):

- Entries between `lt` and `gt` equal to partition item `v`
- No larger entries are to the left of the `lt` index
- No smaller items are to the right of the `gt` index

The idea behind the algorithm:

- Let `v` be the partitioning item `a[lo]`
- Scan `i` from left to right
- `a[i] < v`: exchange `a[lt]` with `a[i]`, increment both `lt` and `i`
- `a[i] > v`: exchange `a[gt]` with `a[i]`, decrement `gt`
- `a[i] == v`: increment `i`

The **lower bound**: If there are $n$ distinct keys and the $i^{th}$ one occurs $x_i$ times, any compare-based sorting algorithm must use at least

$$
\lg \Bigg{(} \frac{N!}{x_1! x_2! \cdots x_n!} \Bigg{)} \text{ ~ } - \displaystyle \sum_{i=1}^n x_i \lg \frac{x_i}{N}
$$

compares in the worst case.

The proposition is that quicksort with 3-way partitioning is **entropy-optimal**.

The bottom line is that randomized quicksort with 3-way partitioning reduces running time from linearithmic to linear in a broad class of applications.

In [49]:
def three_way_qsort(arr, lo=0, hi=None):
    """
    Implements a 3-way partitioning quicksort to account for duplicate keys
    Inputs: arr is a list, lo and hi are integers representing indices
    Output: None, sort done in place
    """
    if hi is None:
        hi = len(arr) - 1
    if hi <= lo:
        return
    lt, gt = lo, hi
    pivot = arr[lo]
    i = lo
    
    while i <= gt:
        if arr[i] < pivot:
            arr[i], arr[lt] = arr[lt], arr[i]
            lt += 1
            i += 1
        elif arr[i] > pivot:
            arr[gt], arr[i] = arr[i], arr[gt]
            gt -= 1
        else:
            i += 1
    
    three_way_qsort(arr, lo, lt - 1)
    three_way_qsort(arr, gt + 1, hi)

c = list("QUICKSORTEXAMPLE")
print("Array to sort: {}".format(c))
three_way_qsort(c)
print("Three-way quicksort results: {}".format(c))

Array to sort: ['Q', 'U', 'I', 'C', 'K', 'S', 'O', 'R', 'T', 'E', 'X', 'A', 'M', 'P', 'L', 'E']
Three-way quicksort results: ['A', 'C', 'E', 'E', 'I', 'K', 'L', 'M', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'X']


## System Sorts (Sorting Applications)

Obvious applications of sorting:

- Sort a list of names
- Organize an MP3 library
- Display Google PageRank results
- List RSS feed in reverse chronological order

Problems that become easier once items are in sorted order:

- Find the median
- Identify statistical outliers
- Binary search in a database
- Find duplicates in a mailing list

Not-so-obvious applications

- Data compression
- Computer graphics
- Computational biology
- Load balancing on a parallel computer

In the 1991, quicksort was running slow when it shouldn't have (realized it was running in quadratic time when there were lots of duplicate keys). This led to folks trying to implement the best version of a sort as possible, and make it the system implementation:

- Cutoff to insertion sort for small subarrays
- Uses a 3-way partitioning scheme
- The partitioning items were:
    - Small arrays: middle entry
    - Medium arrays: median of 3
    - Large arrays: Tukey's ninther (median of the median of 3 samples of size 3 each - approximates the median of 9 and uses at most 12 compares). Better than random shuffle

This system sort is widely used in C, C++, Java 6, etc. There are a ton of sorting options, each with its own set of attributes. Usually, the system sort is good enough.


The sorting holy grail is one that has all the good attributes: in place, stable, best case $N$, average case $N \lg N$, and worst case of $N \lg N$.